In [1]:
import os
import logging
from dotenv import load_dotenv
import google.generativeai as genai
from abc import ABC, abstractmethod

load_dotenv()

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

/Users/mahmoud/agent-with-graphiti/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LLMProvider(ABC):
    @abstractmethod
    def generate(self, prompt: str) -> str:
        pass

class GeminiProvider(LLMProvider):
    def __init__(self, model_name: str = "gemini-2.5-flash"):
        genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
        self.model = genai.GenerativeModel(model_name)

    def generate(self, prompt: str) -> str:
        logger.debug(f"Generating response for prompt length: {len(prompt)}")
        max_retries = 3
        for attempt in range(max_retries):
            try:
                logger.debug(f"Gemini API call attempt {attempt + 1}")
                response = self.model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=0,
                        max_output_tokens=500,
                        top_p=0.8,
                        top_k=40
                    )
                )

                if hasattr(response, 'text') and response.text:
                    logger.debug(f"Gemini response received: {response.text[:200]}...")
                    return response.text

                logger.warning(f"Empty response from Gemini (attempt {attempt + 1})")

            except Exception as e:
                logger.error(f"Gemini API error (attempt {attempt + 1}): {e}")
                if attempt == max_retries - 1:
                    fallback_response = '{"thought": "Technical difficulties", "answer": "I\'m experiencing technical issues. Please try again."}'
                    logger.debug(f"Returning fallback response: {fallback_response}")
                    return fallback_response

                import time
                time.sleep(1)

        fallback_response = '{"thought": "Max retries exceeded", "answer": "Connection issues. Please try again later."}'
        logger.debug(f"Max retries exceeded, returning: {fallback_response}")
        return fallback_response


In [6]:
def create_test_prompt(query: str, history: str = "") -> str:
    return f"""You are an expert Customer Service Agent. Analyze the user's query to understand their intent and plan the appropriate response with the provided tools.

Query: {query}
History: {history}

Available tools: SEARCH_PRODUCTS, PLACE_ORDER, GET_ORDER_HISTORY

Respond in JSON format:
{{"thought": "reasoning", "action": {{"name": "TOOL_NAME", "input": "input"}}}}
OR
{{"thought": "reasoning", "answer": "final answer"}}
"""

llm_provider = GeminiProvider()

prompt = create_test_prompt("I want to buy iPhone")
# print(f"PROMPT:\n{prompt}")
# print("\n" + "-" * 30)

response1 = llm_provider.generate(prompt)
print(f"JSON RESPONSE:\n{response1}")

2025-07-10 17:46:38,490 - DEBUG - Generating response for prompt length: 413
2025-07-10 17:46:38,491 - DEBUG - Gemini API call attempt 1
2025-07-10 17:46:40,580 - DEBUG - Gemini response received: ```json
{
  "thought": "The user wants to buy an iPhone. The appropriate tool to help them find iPhones is `SEARCH_PRODUCTS`. The input for this tool should be 'iPhone'.",
  "action": {
    "name": "S...


JSON RESPONSE:
```json
{
  "thought": "The user wants to buy an iPhone. The appropriate tool to help them find iPhones is `SEARCH_PRODUCTS`. The input for this tool should be 'iPhone'.",
  "action": {
    "name": "SEARCH_PRODUCTS",
    "input": "iPhone"
  }
}
```


In [9]:
def create_test_prompt(query: str, history: str = "iphone") -> str:
    return f"""You are an expert Customer Service Agent. Analyze the user's query to understand their intent and plan the appropriate response with the provided tools.

Query: {query}
History: {history}

Available tools: SEARCH_PRODUCTS, PLACE_ORDER, GET_ORDER_HISTORY

Respond in JSON format:
{{"thought": "reasoning", "action": {{"name": "TOOL_NAME", "input": "input"}}}}
OR
{{"thought": "reasoning", "answer": "final answer"}}
"""

llm_provider = GeminiProvider()

prompt = create_test_prompt("15 pro max")
print(f"PROMPT:\n{prompt}")
print("\n" + "-" * 30)

response1 = llm_provider.generate(prompt)
print(f"JSON RESPONSE:\n{response1}")

2025-07-10 17:48:55,274 - DEBUG - Generating response for prompt length: 409
2025-07-10 17:48:55,275 - DEBUG - Gemini API call attempt 1


PROMPT:
You are an expert Customer Service Agent. Analyze the user's query to understand their intent and plan the appropriate response with the provided tools.

Query: 15 pro max
History: iphone

Available tools: SEARCH_PRODUCTS, PLACE_ORDER, GET_ORDER_HISTORY

Respond in JSON format:
{"thought": "reasoning", "action": {"name": "TOOL_NAME", "input": "input"}}
OR
{"thought": "reasoning", "answer": "final answer"}


------------------------------


2025-07-10 17:48:57,268 - DEBUG - Gemini response received: ```json
{
  "thought": "The user is asking about '15 pro max' and the previous context was 'iphone'. This indicates they are likely looking for information about the 'iPhone 15 Pro Max'. The `SEARCH_P...


JSON RESPONSE:
```json
{
  "thought": "The user is asking about '15 pro max' and the previous context was 'iphone'. This indicates they are likely looking for information about the 'iPhone 15 Pro Max'. The `SEARCH_PRODUCTS` tool is appropriate for this query.",
  "action": {
    "name": "SEARCH_PRODUCTS",
    "input": "iPhone 15 Pro Max"
  }
}
```
